In [36]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

In [37]:
df = pd.read_csv('../YieldPrediction/dataset/AgrcultureDataset.csv')

In [51]:
df.head()

,State_Name,District_Name,Season,Crop,Area,Production
0,0,427,1,2,1592,2000.0
1,0,427,1,74,7,1.0
2,0,427,1,95,226,321.0
3,0,427,4,7,335,641.0
4,0,427,4,22,1010,165.0


In [39]:
df.isna().sum()

State_Name       0
District_Name    0
Crop_Year        0
Season           0
Crop             0
Area             0
Production       0
dtype: int64

In [40]:
df.drop('Crop_Year', axis=1, inplace=True)

In [41]:
import re


def prod(row):
    if re.search('^[0-9]+$', row):
        return float(row)
    else:
        return np.NAN

In [42]:
df.Production = df.Production.apply(prod)
df.dropna(inplace=True)

## for mapping the attributes

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233498 entries, 0 to 246090
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   State_Name     233498 non-null  object 
 1   District_Name  233498 non-null  object 
 2   Season         233498 non-null  object 
 3   Crop           233498 non-null  object 
 4   Area           233498 non-null  float64
 5   Production     233498 non-null  float64
dtypes: float64(2), object(4)
memory usage: 12.5+ MB


In [44]:
for i in df.columns:
    if df[i].dtype == 'object':
        print(dict(zip(list(df[i].unique()),
                       list(i for i in range(len(df[i].unique()))))))

{'Andaman and Nicobar Islands': 0, 'Andhra Pradesh': 1, 'Arunachal Pradesh': 2, 'Assam': 3, 'Bihar': 4, 'Chandigarh': 5, 'Chhattisgarh': 6, 'Dadra and Nagar Haveli': 7, 'Goa': 8, 'Gujarat': 9, 'Haryana': 10, 'Himachal Pradesh': 11, 'Jammu and Kashmir ': 12, 'Jharkhand': 13, 'Karnataka': 14, 'Kerala': 15, 'Madhya Pradesh': 16, 'Maharashtra': 17, 'Manipur': 18, 'Meghalaya': 19, 'Mizoram': 20, 'Nagaland': 21, 'Odisha': 22, 'Puducherry': 23, 'Punjab': 24, 'Rajasthan': 25, 'Sikkim': 26, 'Tamil Nadu': 27, 'Telangana ': 28, 'Tripura': 29, 'Uttar Pradesh': 30, 'Uttarakhand': 31, 'West Bengal': 32}
{'NICOBARS': 0, 'NORTH AND MIDDLE ANDAMAN': 1, 'SOUTH ANDAMANS': 2, 'ANANTAPUR': 3, 'CHITTOOR': 4, 'EAST GODAVARI': 5, 'GUNTUR': 6, 'KADAPA': 7, 'KRISHNA': 8, 'KURNOOL': 9, 'PRAKASAM': 10, 'SPSR NELLORE': 11, 'SRIKAKULAM': 12, 'VISAKHAPATANAM': 13, 'VIZIANAGARAM': 14, 'WEST GODAVARI': 15, 'ANJAW': 16, 'CHANGLANG': 17, 'DIBANG VALLEY': 18, 'EAST KAMENG': 19, 'EAST SIANG': 20, 'KURUNG KUMEY': 21, 'LOHI

In [45]:
for col in ['State_Name', 'District_Name', 'Season', 'Crop', 'Area']:
    df[col] = labelencoder.fit_transform(df[col])

In [46]:
from lightgbm import LGBMRegressor as lc

lcr = lc(bagging_fraction=0.4, bagging_freq=0, boosting_type='gbdt',
         class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
         importance_type='split', learning_rate=0.4, max_depth=-1,
         min_child_samples=66, min_child_weight=0.001, min_split_gain=0,
         n_estimators=30, n_jobs=-1, num_leaves=30, objective=None,
         random_state=3475, reg_alpha=0.1, reg_lambda=0.15, silent=True,
         subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [47]:
lcr.fit(df.drop(['Production'], axis=1), df['Production'])

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0


LGBMRegressor(bagging_fraction=0.4, bagging_freq=0, feature_fraction=0.9,
              learning_rate=0.4, min_child_samples=66, min_split_gain=0,
              n_estimators=30, num_leaves=30, random_state=3475, reg_alpha=0.1,
              reg_lambda=0.15)

In [50]:

import pickle

filename = '../YieldPrediction/lightgbm_yield_model.pkl'
pickle.dump(lcr, open(filename, 'wb'))